# PREPARATION

### Folder structure:
- `data`: raw data
- `tmp`: processed data used by this notebook
- `img`: large-scale images (ignored by git)
- `transfer`: shared images for colleagues (*not* ignored by git, use wisely)

### Install required packages and import them

In [ ]:
# After this line run "Kernel" --> "Restart Kernel"
!pip install -r requirements.txt --user

In [ ]:
import math
import numpy as np
import scipy.io
from scipy import signal
import ecgdata
import visualization as viz
import antropy as ant
import neurokit2 as nk
from tqdm.notebook import tqdm as tqdm
from matplotlib import cm
from matplotlib.ticker import FormatStrFormatter
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
print(nk.__version__)

### Configure script
- `.mat` file containing raw ECG
- `abnorm` defining the ECG class of interest LBBB: `2` AF: `4` 
- `method` defining the XAI method of interest, options: Epsilon - `eps`,  Alpha1Beta0 - `ab0`, WSquare - `wsq`, PresetA - `psa`, IGradient - `igr`

In [ ]:
file = "./tmp/PTB_AF.mat"
method = "igr"
abnorm = 4
dataset = "PTB" # choice: PTB, CPSC, TEST

if abnorm == 4:
    abnorm_label = "AF"
    abnorm_ptb = "AFIB"
    color_abnorm = '#267326'
elif abnorm == 2:
    abnorm_label = "LBBB"
    abnorm_ptb = "CLBBB"
    color_abnorm = '#386cb0'
else:
    abnorm_label = "Error"

file_NORM = "./tmp/PTB_"+abnorm_label+"_NORM_0000-2000.mat"
chan_list = ["I", "II", "III", "aVR", "aVL", "aVF", "V1", "V2", "V3", "V4", "V5", "V6" ]
max_duration = 4096

### Load data from .mat file

- `data[raw]`: ECG raw data 
- `data[rel]`: relevances assigned to input by XAI method in shape of `data[rel]`
- `data[rel_linear]`: relevances assigned to input by XAI method in shape of `data[rel]` (linear activation in last NN layer)
- `data[confidence]`: confidence assigned by Neural Network (*not* ground truth!)
- `data[confidence_linear]`: confidence assigned by Neural Network (*not* ground truth! linear activation in last NN layer)
- `data[pid]`: unique patient identifier

Data is ordered by pid, not by cohort.

In [ ]:
data = scipy.io.loadmat(file)

if len(data['raw'])==len(data['rel'])==len(data['rel_linear'])==len(data['confidence'])==len(data['pid']):
    numberSubjects = int(len(data['raw']))
    print("Loaded data of " + str(numberSubjects) + " patients with or without " + str(abnorm_label))
else:
    numberSubjects = int(len(data['raw']))
    print("Error, data dimensions not uniform.")


In [ ]:
# load data of controls (PTB)
if dataset == "PTB":
    dataNORM = scipy.io.loadmat(file_NORM)

    if len(dataNORM['raw'])==len(dataNORM['rel'])==len(dataNORM['rel_linear'])==len(dataNORM['confidence'])==len(dataNORM['pid']):
        numberSubjects += int(len(dataNORM['raw']))
        print("Loaded data of " + str(numberSubjectsNORM) + " patients without " + str(abnorm_label))
    else:
        numberSubjects += int(len(dataNORM['raw']))
        print("Error, data dimensions not uniform.")

### Get cohort identifiers

- `i_abnorm`: IDs of patients with `abnorm`
- `i_sinus`: IDs of healthy subject 

In [ ]:
def getCohortIdx(data, abnorm): #### TODO: Check 0 patients?
  i_tmp, _ = ecgdata.getCohortsCPSC(abnorm)
  i_abnorm = [i for i,val in enumerate(data['pid']) if int(val[1:])-1 in i_tmp]
  i_sinus = [i for i,val in enumerate(data['pid']) if i not in i_abnorm]
  return i_abnorm, i_sinus
    
if dataset == "CPSC" or dataset == "TEST":
    i_abnorm, i_sinus = getCohortIdx(data, abnorm)

if dataset == "PTB":
    i_abnorm = [i for i in range(len(data['raw']))]
    i_sinus = [i for i in range(len(data['raw']), numberSubjects)]

print("There are " + str(len(i_abnorm)) + " patients in the dataset for " + str(abnorm_label) + " and " + str(len(i_sinus)) + " healthy controls")

## Preprocess relevance data

In [ ]:
# get data - two cohorts (CPSC)
if dataset == "CPSC":
    relevances = np.concatenate( (data['rel_linear'][i_abnorm], data['rel_linear'][i_sinus] ))
    relevances_sigmoid = np.concatenate( (data['rel'][i_abnorm], data['rel'][i_sinus] ))
    raw_data = np.concatenate( (data['raw'][i_abnorm], data['raw'][i_sinus] ))
    conf = np.concatenate( [data['confidence'][i_abnorm], data['confidence'][i_sinus] ] )
    conf_lin = np.concatenate( [data['confidence_linear'][i_abnorm], data['confidence_linear'][i_sinus] ] )
    pid = np.concatenate( [data['pid'][i_abnorm], data['pid'][i_sinus] ] )
    type = np.concatenate( (np.zeros(int(numberSubjects/2)), np.ones(int(numberSubjects/2)) ) ) # 0 abnorm 1 i_sinus

In [ ]:
# get data - two cohorts - separate files (PTB)
if dataset == "PTB":
    relevances = np.concatenate( (data['rel_linear'], dataNORM['rel_linear'] ))
    relevances_sigmoid = np.concatenate( (data['rel'], dataNORM['rel'] ))
    raw_data = np.concatenate( (data['raw'], dataNORM['raw'] ))
    conf = np.concatenate( [data['confidence'], dataNORM['confidence'] ] )
    conf_lin = np.concatenate( [data['confidence_linear'], dataNORM['confidence_linear'] ] )
    pid_abnorm, pid_normal = ecgdata.getCohortsPTB(abnorm_ptb)
    pid = np.concatenate( (np.array(pid_abnorm.values), np.array(pid_normal[:2000].values)) )
    type = np.concatenate( (np.zeros(int(len(i_abnorm))), np.ones(int(len(i_sinus))) ) ) # 0 abnorm 1 i_normal

In [ ]:
# get data - one cohort (Ribeiro Test Data)
if dataset == "TEST":
    relevances = data['rel_linear']
    raw_data = data['raw']
    conf = data['confidence']
    pid = data['pid'][0]
    type = np.zeros(int(numberSubjects)) # 0 abnorm 1 i_sinus

In [ ]:
# for given method: normalize values to -1,1 for each ecg, keeping 0 as center
rel_normalized = np.zeros([numberSubjects,4096,12]) 
rel_lin_normalized = np.zeros([numberSubjects,4096,12])
raw_normalized = np.zeros([numberSubjects,4096,12]) 
for patient in range(len(relevances_sigmoid)):
    rel_normalized[patient] = relevances_sigmoid[patient]/np.max(np.abs(relevances_sigmoid[patient]))
    rel_lin_normalized[patient] = relevances[patient]/np.max(np.abs(relevances[patient]))
    raw_normalized[patient] = raw_data[patient]/np.max(np.abs(raw_data[patient]))

## Test results with Ribeiro thresholds

In [ ]:
abnorm_pos = []
threshold = np.array([0.124, 0.07, 0.05, 0.278, 0.390, 0.174])

for i in range(numberSubjects):
  if conf[i][abnorm]>=threshold[abnorm]:
    abnorm_pos.append(i)
    
print("Detected " + str(len(abnorm_pos)) + " patients larger than the threshold:" + str(abnorm_pos))

In [ ]:
print(np.min(relevances[0:200]))
print(np.max(relevances[0:200]))
print(np.min(relevances[200:400]))
print(np.max(relevances[200:400]))
print(np.where(pid == 'A0977'))
print(conf[np.where(pid == 'A0977')])
print(np.where(pid == 'A1017'))
print(conf[np.where(pid == 'A1017')])

# FIGURES

## Qualitative Analysis: 12 Channel Visualization and Heatmap

In [ ]:
# Create relevances plots and heatmaps
for patient in tqdm(range(200,400)):
  #viz.plotRelevances(raw_normalized[patient], "linear", rel_lin_normalized[patient], method, data['pid'][patient])
  #viz.plotRelevances12(raw_data[patient], "sigmoid", rel_normalized[patient], method, data['pid'][patient])
  viz.plotRelevances12(raw_data[patient], "linear", rel_lin_normalized[patient], method, pid[patient])
  #viz.activationHeatmap(rel_normalized[patient], rel_lin_normalized[patient], method, data['pid'][patient])

In [ ]:
plt.rcParams.update({'font.size': 18})

viz.plotRelevances12Normed(raw_data[27][:2001], "linear", rel_lin_normalized[27][:2001], method, pid[27])
#viz.plotRelevances12(raw_data[172][:2001], "linear", rel_lin_normalized[172][:2001], method, pid[172])

## Analysis 1: Relevances vs. DNN Confidence

In [ ]:
# compute FN/FP
idx_false_positive = np.intersect1d( np.where(type==1) , abnorm_pos )
idx_false_negative = np.setdiff1d( np.where(type==0), abnorm_pos )

# prepare matrices for plots
matSumRelevances = np.zeros((numberSubjects,12))
matMeanRelevances = np.zeros((numberSubjects,12))
matEntropyRelevances = np.zeros((numberSubjects,12))
matFourierMaxima = np.zeros((numberSubjects,12))
matCertainty = np.zeros(numberSubjects)
matCertainty_linear = np.zeros(numberSubjects)

# iterate over data and compute results
for patient in range(numberSubjects):
    matMeanRelevances[patient] = np.mean(relevances[patient,:,:].flatten())
    matCertainty[patient] = conf[patient, abnorm]
    matCertainty_linear[patient] = conf_lin[patient, abnorm]  
        
def sigmoid(x):
  return 1 / (1 + np.exp(-x))
def sigmoid_reverse(x):
  return np.log(x / (1 - x))

In [ ]:
print("Min Mean Abnorm: " + str(np.min(np.min(matMeanRelevances,1)[matCertainty>=threshold[abnorm]])))
print("Max Mean Abnorm: " + str(np.max(np.max(matMeanRelevances,1)[matCertainty>=threshold[abnorm]])))
print("Median Mean Abnorm: " + str(np.median(matMeanRelevances[matCertainty>=threshold[abnorm]].flatten())))
print("Median Mean Sinus: " + str(np.median(matMeanRelevances[matCertainty<threshold[abnorm]].flatten())))
print("Min Mean Sinus: " + str(np.min(np.min(matMeanRelevances,1)[matCertainty<threshold[abnorm]])))
print("Max Mean Sinus: " + str(np.max(np.max(matMeanRelevances,1)[matCertainty<threshold[abnorm]])))
print(matMeanRelevances[0])
print(np.mean(matMeanRelevances,1)[idx_false_negative])

### Histogram
All relevances of all ECGs in one plot.

In [ ]:
plt.rcParams.update({'font.size': 18})
if dataset == "":#"PTB":
    plt.rcParams["figure.figsize"] = (10,6)
    fig, (ax2, ax) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [0.5, 1]})
else:
    plt.rcParams["figure.figsize"] = (25,20)
    fig = plt.figure()
    ax = fig.add_subplot(4, 2, 1)

print("Min AF: " + str(np.min(relevances[:200])))
print("Min Sinus " + str(np.min(relevances[len(i_abnorm):len(i_abnorm)+200])))
print("Max AF: " + str(np.max(relevances[:200])))
print("Max Sinus " + str(np.max(relevances[len(i_abnorm):len(i_abnorm)+200])))
print("Mean AF: " + str(np.mean(relevances[:len(i_abnorm)])))
print("Mean Sinus " + str(np.mean(relevances[len(i_abnorm):len(i_abnorm)+len(i_abnorm)])))
print("Std AF " + str(np.std(relevances[:len(i_abnorm)])))
print("Std Sinus: " + str(np.std(relevances[len(i_abnorm):len(i_abnorm)+len(i_abnorm)])))
data_rel = relevances[:200].flatten()
data_rel2 = relevances[len(i_abnorm):len(i_abnorm)+200].flatten()

if dataset == "PTB":
    data_CPSC = scipy.io.loadmat("./tmp/200each_"+abnorm_label+".mat")
    i_abnorm_CPSC, i_sinus_CPSC = getCohortIdx(data_CPSC, abnorm)
    data_CPSC_rel = data_CPSC['rel_linear'][i_abnorm_CPSC].flatten()
    data_rel2 = data_CPSC['rel_linear'][i_abnorm_CPSC][:200].flatten()
    data_CPSC_rel2 = data_CPSC['rel_linear'][i_sinus_CPSC].flatten()
    #ax.invert_yaxis()

    
plt.xlim(-0.2,0.2)
#ax.hist(data_rel2, bins=600, log=True, color="gray", alpha=0.8)
#ax.hist(data_rel, bins=120, log=True, color=('#267326' if abnorm == 4 else '#386cb0'), alpha=0.5)
ax.hist(data_rel2, bins=600, log=True, color="#998ec3", alpha=0.9)
ax.hist(data_rel, bins=200, log=True, color='#f1a340', alpha=0.7)
plt.grid(True)
plt.xlabel("Relevances $R_{n,j,k}$"); 

if dataset == "":#"PTB":
    ax.text(0.985, 0.1, 'PTB-XL', horizontalalignment='right', verticalalignment='bottom', transform=ax.transAxes)
    ax2.text(0.97, 0.9, 'CPSC', horizontalalignment='right', verticalalignment='top', transform=ax2.transAxes)
    ax2.hist(data_CPSC_rel2, bins=500, log=True, color="gray", alpha=0.8)
    ax2.hist(data_CPSC_rel, bins=500, log=True, color=('#267326' if abnorm == 4 else '#386cb0'), alpha=0.5)
    ax2.set_xlim(-0.2,0.2)
    ax2.set_xticks([-0.2,-0.15,-0.1,-0.05,0,0.05,0.1,0.15,0.2],[])
    ax2.grid(True)
    plt.ylabel("$~~~~~~~~~~~~~~~~~~~~$Frequency"); 
else:
    plt.ylabel("Frequency"); 

plt.tight_layout(pad=-0.3)
#plt.legend(["Normal",abnorm_label])
plt.legend(["CPSC AF","PTB-XL AF"])

plt.savefig('./img/hist_all_subjects_' + abnorm_label + '.pdf', bbox_inches='tight')

### Boxplot
One relevance boxplot for each ECG. Spaced linearly (x-axis above), with sigmoid confidence on x-axis below.

In [ ]:
 # plot settings
plt.rcParams.update({'font.size': 18})
plt.rcParams["figure.figsize"] = (30,20)

fig = plt.figure()
ax = fig.add_subplot(4, 2, 1)

medianprops = dict(linestyle='-', linewidth=4)
meanpointprops = dict(marker='o', markersize=6, markeredgecolor='black', markerfacecolor='gray')
meanpointprops2 = dict(marker='o', markersize=6, markeredgecolor='black', markerfacecolor=('#267326' if abnorm == 4 else '#386cb0'))

for patient in range(len(i_abnorm), len(i_abnorm)+200):
    bp0 = ax.boxplot(relevances[patient,:,:].flatten(), meanprops=meanpointprops, showmeans=True, positions=[matCertainty_linear[patient]], widths=0.05, patch_artist=True, sym='',zorder=1)
    for item in ['boxes', 'whiskers','medians', 'fliers', 'caps']:
            plt.setp(bp0[item], color="gray")
    plt.setp(bp0["medians"], color="gray")
    
for patient in range(200):
    bp0 = ax.boxplot(relevances[patient,:,:].flatten(), medianprops=medianprops, meanprops=meanpointprops2, showmeans=True,positions=[matCertainty_linear[patient]], widths=0.05, patch_artist=True, sym='',zorder=1)
    for item in ['boxes', 'whiskers','medians', 'fliers', 'caps']:
            if abnorm == 4:
                plt.setp(bp0[item], color="#267326")
            else:
                plt.setp(bp0[item], color="#386cb0")
    plt.setp(bp0["medians"], color="black")

for patient in idx_false_negative:
    bp0 = ax.boxplot(relevances[patient,:,:].flatten(), medianprops=medianprops, meanprops=meanpointprops, showmeans=True, positions=[matCertainty_linear[patient]], widths=0.05, patch_artist=True, sym='',zorder=1)
    for item in ['boxes', 'whiskers','medians', 'fliers', 'caps']:
            plt.setp(bp0[item], color="gray")
    plt.setp(bp0["medians"], color="black")

plt.plot( [sigmoid_reverse(0.39 if abnorm == 4 else 0.05), sigmoid_reverse(0.39 if abnorm == 4 else 0.05)], [-0.5, 0.5], color='gray', linestyle='--', linewidth=4, label=abnorm_label + " threshold")   # use threshold variable
plt.scatter(matCertainty_linear[idx_false_positive], np.mean(matMeanRelevances,1)[idx_false_positive], label="False Positive", s=20, color="red",marker="x")
plt.scatter(matCertainty_linear[idx_false_negative], np.mean(matMeanRelevances,1)[idx_false_negative], label="False Negative", s=30, color="red", marker="x",zorder=2)

ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
grey_patch = mpatches.Patch(color='grey', label='DNN Classification: No ' + abnorm_label)
blue_patch = mpatches.Patch(color=('#267326' if abnorm == 4 else '#386cb0'), label='DNN Classification: ' + abnorm_label)
line_patch = mlines.Line2D([0], [0], color='gray', linestyle='--', linewidth=3, label=abnorm_label + " threshold")
x_patch = mlines.Line2D([0], [0], color='red', marker='x', linestyle='', label="False Negative")

plt.grid(True)
plt.yticks(fontsize=12)
plt.ylim([-0.0035,0.0035])
plt.ylabel("Relevances $R_n$"); 
plt.legend(handles=[blue_patch, grey_patch, line_patch, x_patch], loc="lower right", prop={'size': 12})

# sigmoid axis below
plt.xlabel("Probability of " + abnorm_label + " being present according to DNN $C_n$");
plt.xticks([np.min(matCertainty_linear), 0, np.max(matCertainty_linear)], [round(sigmoid(np.min(matCertainty_linear)),2), sigmoid(0), round(sigmoid(np.max(matCertainty_linear)),2)], fontsize=12)

# linear axis above
ax2 = ax.twiny()
ax2.set_xlim(ax.get_xlim())
ax2.set_xticks(range(math.floor(np.min(matCertainty_linear)),math.ceil(np.max(matCertainty_linear))+1,1))
plt.xticks(fontsize=12)

plt.savefig('./img/boxplot_all_subjects_' + abnorm_label + '.pdf', bbox_inches='tight')

### Scatter Plot
Mean of relevances of each ECG vs. DNN confidence (sigmoid).

In [ ]:
# plot settings
plt.rcParams.update({'font.size': 12})
plt.rcParams["figure.figsize"] = (8, 4)

fig = plt.figure()
plt.scatter(matCertainty[matCertainty>=threshold[abnorm]], np.sum(matMeanRelevances,1)[matCertainty>=threshold[abnorm]], label="DNN classification: "+abnorm_label,marker="o", edgecolors='black', color="#267326", s=60, alpha=0.8)
plt.scatter(matCertainty[matCertainty<threshold[abnorm]], np.sum(matMeanRelevances,1)[matCertainty<threshold[abnorm]], label="DNN classification: no "+abnorm_label, color="gray", edgecolors='black', s=60, alpha=0.8)
#plt.scatter(matCertainty[idx_false_positive], np.sum(matSumRelevances,1)[idx_false_positive], label="False Positive", s=20, color="red",marker="x")
plt.scatter(matCertainty[idx_false_negative], np.sum(matMeanRelevances,1)[idx_false_negative], label="False Negative ", s=20, color="red", marker="x")
plt.xlabel("Probability of "+abnorm_label+" being present according to DNN $C_n$"); plt.ylabel("Mean of relevances $M_n$",fontsize=12); 
plt.plot( [threshold[abnorm], threshold[abnorm]], [-30, 30], color='gray', linestyle='--', linewidth=2, label=abnorm_label+" threshold")   # use threshold variable
plt.legend(loc="lower right", prop={'size': 10})
plt.grid(True)
#plt.yticks(np.arange(-18, 18, 4.0))
plt.ylim([-0.006,0.004])
plt.xlim([-0.05,1.05])

plt.savefig('./img/Conf_vs_Mean_'+abnorm_label+'.pdf', bbox_inches='tight')

## Analysis 2: Differences between Leads

In [ ]:
plt.rcParams.update({'font.size': 12})
#relevancesAF = data['rel_linear'][i_abnorm]
#relevancesSinus = data['rel_linear'][i_sinus]
relevancesAF = data['rel_linear']
relevancesSinus = dataNORM['rel_linear']
xAF = np.sum(relevancesAF, axis=1)
xNormal = np.sum(relevancesSinus, axis=1)

print("Median AF: " + str(np.median(xAF)))
print("Median Sinus: " + str(np.median(xNormal)))
print("Min/Max AF: " + str(np.min(relevancesAF)) + " " + str(np.max(relevancesAF)))
print("Min/Max Sinus: " + str(np.min(relevancesSinus)) + " " + str(np.max(relevancesSinus)))

viz.plotRelevanceBoxplot(relevancesAF[:200], relevancesSinus[:200], abnorm_label, "Normal", "linear", method)
#viz.plotEntropyBoxplot(relevancesAF, relevancesSinus, abnorm_label, "Normal", "linear", method)

## Analysis 3: Average Heart Beat vs Average Relevances per Heart Beat

In [ ]:
def getSegments(patient, lead):
    ecg = np.swapaxes(raw_data,1,2)[patient][11]
    ecg_cleaned = nk.ecg_clean(ecg, sampling_rate=400)
    return nk.epochs_to_df(nk.ecg_segment(ecg_cleaned, rpeaks=None, sampling_rate=400, show=False))

def getPatientAvgs(patient, lead):
    segments = getSegments(patient, lead)
    result = segments.groupby('Label').agg({'Index': ['min', 'max']})
    beats = len(result["Index"])
    duration = int(len(segments)/beats)
    avg_beat = np.zeros(duration)
    avg_rel = np.zeros(duration)
    all_beat = []
    all_rel = []
    # sum values of all beats
    for idx in range(beats):
        start = result["Index"]["min"][idx]
        end = result["Index"]["max"][idx]
        # segments always have same length, even if exceeding number of samples
        if end >= max_duration:
            zero_padding = np.zeros((end-max_duration+1))
            avg_beat += np.concatenate([raw_data[patient][start:end+1, lead], zero_padding])
            avg_rel += np.concatenate([relevances[patient][start:end+1, lead], zero_padding])
            all_beat.append(np.concatenate([raw_data[patient][start:end+1, lead], zero_padding]))
            all_rel.append(np.concatenate([relevances[patient][start:end+1, lead], zero_padding]))
        elif start < 0:
            zero_padding = np.zeros(int(-start))
            avg_beat += np.concatenate([zero_padding, raw_data[patient][:end+1, lead]])
            avg_rel += np.concatenate([zero_padding, relevances[patient][:end+1, lead]])
            all_beat.append(np.concatenate([zero_padding, raw_data[patient][:end+1, lead]]))
            all_rel.append(np.concatenate([zero_padding, relevances[patient][:end+1, lead]]))
        else:
            avg_beat += raw_data[patient][start:end+1, lead]
            avg_rel += relevances[patient][start:end+1, lead]
            all_beat.append(raw_data[patient][start:end+1, lead])
            all_rel.append(relevances[patient][start:end+1, lead])
    return avg_beat, avg_rel, all_beat, all_rel, duration

def addAvgs(all_avg_beats, all_avg_rels, avg_beat, avg_rel):
    diff = len(all_avg_beats) - len(avg_beat)
    if diff < 0:
        c = avg_beat.copy()
        c[int(-diff/2):len(all_avg_beats)+(int(-diff/2))] += all_avg_beats
        d = avg_rel.copy()
        d[int(-diff/2):len(all_avg_rels)+(int(-diff/2))] += all_avg_rels
    else:
        c = all_avg_beats.copy()
        c[int(diff/2):len(avg_beat)+(int(diff/2))] += avg_beat
        d = all_avg_rels.copy()
        d[int(diff/2):len(avg_rel)+(int(diff/2))] += avg_rel
    return c,d

### Plots

- plotAvgsPatient: average beat for all leads and one patient
- plotAvgs: average beat for all leads and range of patients
- plotAvgsLead: average beat for one lead and two cohorts, plotRange = steps of averaged patients to plot

In [ ]:
def plotAvgsPatient(patient, title):
    fig, axs = plt.subplots(4,3,figsize = (20, 15))
    max_avg_beat = 0
    max_avg_rel = 0
    max_all_beat = 0
    max_all_rel = 0
    for lead in range(12):
        avg_beat, avg_rel, all_beat, all_rel, duration = getPatientAvgs(patient, lead)
        max_avg_beat = np.max([np.max(np.abs(avg_beat)), max_avg_beat])
        max_avg_rel = np.max([np.max(np.abs(avg_rel)), max_avg_rel])
        max_all_beat = np.max([np.max(np.abs(all_beat)), max_all_beat])
        max_all_rel = np.max([np.max(np.abs(all_rel)), max_all_rel])
    fig.suptitle("Average Heart Beats | " + title)
    for lead in range(12):
        avg_beat, avg_rel, all_beat, all_rel, duration = getPatientAvgs(patient, lead)
        # normalize to -1,1
        avg_beat /= max_avg_beat
        avg_rel /= max_avg_rel
        all_beat /= max_all_beat
        all_rel /= max_all_rel
        # plot raw data and relevances
        x_range = np.arange(0, duration, 1)
        plt.subplot(4,3,lead+1)
        plt.title(chan_list[lead])
        plt.boxplot(all_rel, showfliers=False)
        plt.xticks([])
        plt.plot(x_range, avg_beat, color="blue", label="ECG") # TODO
    #    plt.plot(x_range, avg_rel, color="black", label="Relevances")
        plt.ylim(-1,1)
        plt.legend()
    plt.tight_layout()
    plt.savefig('./img/boxplot_beats_' + str(pid[patient]) + '_' + abnorm_label + '.pdf')
    #plt.show()
    plt.close()
    
def plotAvgs(patientRange, name):
    fig, axs = plt.subplots(4,3,figsize = (40, 30))
    fig.suptitle("Average Heart Beats | " + name)
    all_beats = []
    all_rels = []
    for lead in range(12):
        all_avg_beats = np.zeros(200)
        all_avg_rels = np.zeros(200)
        for patient in tqdm(patientRange):
            try:
                max_avg_beat = 0
                max_avg_rel = 0
                max_all_beat = 0
                max_all_rel = 0
                for lead2 in range(12):
                    avg_beat, avg_rel, all_beat, all_rel, duration = getPatientAvgs(patient, lead2)
                    max_avg_beat = np.max([np.max(np.abs(avg_beat)), max_avg_beat])
                    max_avg_rel = np.max([np.max(np.abs(avg_rel)), max_avg_rel])
                    max_all_beat = np.max([np.max(np.abs(all_beat)), max_all_beat])
                    max_all_rel = np.max([np.max(np.abs(all_rel)), max_all_rel])
                avg_beat, avg_rel, all_beat, all_rel, duration = getPatientAvgs(patient, lead)
                # normalize to -1,1
                avg_beat /= max_avg_beat
                avg_rel /= max_avg_rel
                all_beat /= max_all_beat
                all_rel /= max_all_rel
            except:
                print(str(pid[patient]) + " and lead " + str(lead) + " not included due to error.")
                continue
            # copy shorter beat on bigger one
            all_avg_beats, all_avg_rels = addAvgs(all_avg_beats, all_avg_rels, avg_beat, avg_rel)
        all_beats.append(all_avg_beats)
        all_rels.append(all_avg_rels)
    # normalize to -1,1
    all_beats /= np.max(np.abs(all_beats))
    all_rels /= np.max(np.abs(all_rels))
    # plot raw data and relevances
    for lead in range(12):
        x_range = np.arange(0, len(all_beats[lead]), 1)
        plt.subplot(4,3,lead+1)
        plt.title(chan_list[lead])
        #plt.boxplot(all_rel, showfliers=False)
        plt.xticks([])
        plt.plot(x_range, all_beats[lead], color="blue", label="ECG") # TODO
        plt.plot(x_range, all_rels[lead], color="black", label="Relevances")
        plt.ylim(-1,1)
        plt.legend()
    plt.tight_layout()
    plt.savefig('./img/beats_' + name + '.pdf')
    #plt.show()
    plt.close()

In [ ]:
plotAvgs(range(len(i_abnorm)), abnorm_label)
plotAvgs(range(len(i_abnorm),len(i_sinus)), "healthy_" + abnorm_label)

In [ ]:
# plot single patient
patient = 200
title = str(abnorm_label) + ", " + str(conf[patient][abnorm]*100) + "%"
plotAvgsPatient(patient, title)

### Image: Average Heart Beat with Variance

In [ ]:
def plotAvgsLeadImage(patientRange1, patientRange2, name, lead):
    all_beats1 = []
    all_rels1 = []
    all_beats2 = []
    all_rels2 = []
    steps_beats1 = []
    steps_rels1 = []
    steps_beats2 = []
    steps_rels2 = []
    # first patientRange
    for patient in tqdm(patientRange1):
        try:
            max_avg_beat = 0
            max_avg_rel = 0
            max_all_beat = 0
            max_all_rel = 0
            for lead2 in range(12):
                avg_beat, avg_rel, all_beat, all_rel, duration = getPatientAvgs(patient, lead2)
                max_avg_beat = np.max([np.max(np.abs(avg_beat)), max_avg_beat])
                max_avg_rel = np.max([np.max(np.abs(avg_rel)), max_avg_rel])
                max_all_beat = np.max([np.max(np.abs(all_beat)), max_all_beat])
                max_all_rel = np.max([np.max(np.abs(all_rel)), max_all_rel])
            avg_beat, avg_rel, all_beat, all_rel, duration = getPatientAvgs(patient, lead)
            # normalize to -1,1
            avg_beat /= max_avg_beat
            avg_rel /= max_avg_rel
            all_beat /= max_all_beat
            all_rel /= max_all_rel
        except:
            print(str(pid[patient]) + " and lead " + str(lead) + " not working, added zeros to average.")
            avg_beat = np.zeros(10)
            avg_rel = np.zeros(10)
            all_beat = np.zeros(10)
            all_rel = np.zeros(10)
        steps_beats1.append(avg_beat)
        steps_rels1.append(avg_rel)
    # second patientRange
    for patient in tqdm(patientRange2):
        try:
            max_avg_beat = 0
            max_avg_rel = 0
            max_all_beat = 0
            max_all_rel = 0
            for lead2 in range(12):
                avg_beat, avg_rel, all_beat, all_rel, duration = getPatientAvgs(patient, lead2)
                max_avg_beat = np.max([np.max(np.abs(avg_beat)), max_avg_beat])
                max_avg_rel = np.max([np.max(np.abs(avg_rel)), max_avg_rel])
                max_all_beat = np.max([np.max(np.abs(all_beat)), max_all_beat])
                max_all_rel = np.max([np.max(np.abs(all_rel)), max_all_rel])
            avg_beat, avg_rel, all_beat, all_rel, duration = getPatientAvgs(patient, lead)
            # normalize to -1,1
            avg_beat /= max_avg_beat
            avg_rel /= max_avg_rel
            all_beat /= max_all_beat
            all_rel /= max_all_rel
        except:
            print(str(pid[patient]) + " and lead " + str(lead) + " not working, added zeros to average.")
            avg_beat = np.zeros(10)
            avg_rel = np.zeros(10)
            all_beat = np.zeros(10)
            all_rel = np.zeros(10)
        steps_beats2.append(avg_beat)
        steps_rels2.append(avg_rel)
    # zeropadding for same sample size
    max_len = np.max([max(len(x) for x in steps_beats1),max(len(x) for x in steps_beats2)])
    for i,step in enumerate(steps_beats1):
        pad1 = np.zeros(math.floor((max_len-len(step))/2))
        pad2 = np.zeros(math.ceil((max_len-len(step))/2))
        steps_beats1[i] = np.concatenate([pad1, step, pad2])
    for i,step in enumerate(steps_beats2):
        pad1 = np.zeros(math.floor((max_len-len(step))/2))
        pad2 = np.zeros(math.ceil((max_len-len(step))/2))
        steps_beats2[i] = np.concatenate([pad1, step, pad2])
    for i,step in enumerate(steps_rels1):
        pad1 = np.zeros(math.floor((max_len-len(step))/2))
        pad2 = np.zeros(math.ceil((max_len-len(step))/2))
        steps_rels1[i] = np.concatenate([pad1, step, pad2])
    for i,step in enumerate(steps_rels2):
        pad1 = np.zeros(math.floor((max_len-len(step))/2))
        pad2 = np.zeros(math.ceil((max_len-len(step))/2))
        steps_rels2[i] = np.concatenate([pad1, step, pad2])
    # calculate mean for every sample
    all_beats1 = np.mean(steps_beats1, axis=0)
    all_beats2 = np.mean(steps_beats2, axis=0)
    all_rels1 = np.mean(steps_rels1, axis=0)
    all_rels2= np.mean(steps_rels2, axis=0)
    # normalize to -1,1
    max_beats = np.max([np.max(np.abs(all_beats1)),np.max(np.abs(all_beats2))])
    max_rels = np.max([np.max(np.abs(all_rels1)),np.max(np.abs(all_rels2))])
    all_beats1 /= max_beats
    all_rels1 /= max_rels
    all_beats2 /= max_beats
    all_rels2 /= max_rels
    ##### plot raw data and relevances #####
    fig, axs = plt.subplots(2,2,figsize = (12, 8))
    fig.suptitle(abnorm_label + " classification | Lead " + chan_list[lead], fontsize=16)
    factor = 5 # upsampling factor for relevances in scatter plot
    # first patientRange
    plt.subplot(2,2,1)
    resampled = signal.resample_poly(all_rels1, factor, 1)
    padding = (max_len-len(all_beats1))/2
    x_range = np.arange(padding, len(all_beats1)+padding, 1)
    sigma1 = np.var(steps_rels1, axis=0)/max_rels
    plt.plot(x_range, all_beats1, color="black", label=abnorm_label+" ECG")
    plt.scatter(np.arange(padding, len(resampled)/factor+padding, 1/factor), resampled, c=resampled, cmap=cm.coolwarm, s=20, vmin=-1, vmax=1)
    plt.xticks([])
    plt.yticks(fontsize=12)
    plt.ylim(-1.05,1.05)
    plt.xlim(0,max_len)
    plt.legend()
    plt.subplot(2,2,2)
    plt.fill_between(x_range, all_rels1+sigma1, all_rels1-sigma1, facecolor='orange', alpha=0.5)
    plt.scatter(np.arange(padding, len(resampled)/factor+padding, 1/factor), resampled, c=resampled, cmap=cm.coolwarm, s=1, vmin=-1, vmax=1)
    plt.xticks([])
    plt.yticks(fontsize=12)
    plt.ylim(-1.05,1.05)
    plt.xlim(0,max_len)
    # second patientRange
    plt.subplot(2,2,3)
    labels = np.arange(0, max_len/400, 0.1)
    resampled = signal.resample_poly(all_rels2, factor, 1)
    padding = (max_len-len(all_beats2))/2
    x_range = np.arange(padding, len(all_beats2)+padding, 1)
    sigma2 = np.var(steps_rels2, axis=0)/max_rels
    plt.plot(x_range, all_beats2, color="black", label="Normal ECG")
    plt.scatter(np.arange(padding, len(resampled)/factor+padding, 1/factor), resampled, c=resampled, cmap=cm.coolwarm, s=20, vmin=-1, vmax=1)
    plt.xticks(np.arange(0, max_len, 400*0.1), [str(round(float(label), 2)) for label in labels], fontsize=12)
    plt.yticks(fontsize=12)
    plt.ylim(-1.05,1.05)
    plt.xlim(0,max_len)
    plt.xlabel("seconds")
    plt.legend()
    plt.subplot(2,2,4)
    plt.fill_between(x_range, all_rels2+sigma2, all_rels2-sigma2, facecolor='orange', alpha=0.5)
    plt.scatter(np.arange(padding, len(resampled)/factor+padding, 1/factor), resampled, c=resampled, cmap=cm.coolwarm, s=1, vmin=-1, vmax=1)
    plt.xticks(np.arange(0, max_len, 400*0.1), [str(round(float(label), 2)) for label in labels], fontsize=12)
    plt.yticks(fontsize=12)
    plt.ylim(-1.05,1.05)
    plt.xlim(0,max_len)
    plt.xlabel("seconds")
    # save figure
    plt.tight_layout()
    plt.savefig('./img/beats_' + name + '.pdf')
    plt.close()

In [ ]:
def plotAvgsLeadImageComparison(patientRange1, patientRange2, name, lead):
    all_beats1 = []
    all_rels1 = []
    all_beats2 = []
    all_rels2 = []
    steps_beats1 = []
    steps_rels1 = []
    steps_beats2 = []
    steps_rels2 = []
    # first patientRange
    for patient in tqdm(patientRange1):
        try:
            max_avg_beat = 0
            max_avg_rel = 0
            max_all_beat = 0
            max_all_rel = 0
            for lead2 in range(12):
                avg_beat, avg_rel, all_beat, all_rel, duration = getPatientAvgs(patient, lead2)
                max_avg_beat = np.max([np.max(np.abs(avg_beat)), max_avg_beat])
                max_avg_rel = np.max([np.max(np.abs(avg_rel)), max_avg_rel])
                max_all_beat = np.max([np.max(np.abs(all_beat)), max_all_beat])
                max_all_rel = np.max([np.max(np.abs(all_rel)), max_all_rel])
            avg_beat, avg_rel, all_beat, all_rel, duration = getPatientAvgs(patient, lead)
            # normalize to -1,1
            avg_beat /= max_avg_beat
            avg_rel /= max_avg_rel
            all_beat /= max_all_beat
            all_rel /= max_all_rel
        except:
            print(str(pid[patient]) + " and lead " + str(lead) + " not working, added zeros to average.")
            avg_beat = np.zeros(10)
            avg_rel = np.zeros(10)
            all_beat = np.zeros(10)
            all_rel = np.zeros(10)
        steps_beats1.append(avg_beat)
        steps_rels1.append(avg_rel)
    # second patientRange
    for patient in tqdm(patientRange2):
        try:
            max_avg_beat = 0
            max_avg_rel = 0
            max_all_beat = 0
            max_all_rel = 0
            for lead2 in range(12):
                avg_beat, avg_rel, all_beat, all_rel, duration = getPatientAvgs(patient, lead2)
                max_avg_beat = np.max([np.max(np.abs(avg_beat)), max_avg_beat])
                max_avg_rel = np.max([np.max(np.abs(avg_rel)), max_avg_rel])
                max_all_beat = np.max([np.max(np.abs(all_beat)), max_all_beat])
                max_all_rel = np.max([np.max(np.abs(all_rel)), max_all_rel])
            avg_beat, avg_rel, all_beat, all_rel, duration = getPatientAvgs(patient, lead)
            # normalize to -1,1
            avg_beat /= max_avg_beat
            avg_rel /= max_avg_rel
            all_beat /= max_all_beat
            all_rel /= max_all_rel
        except:
            print(str(pid[patient]) + " and lead " + str(lead) + " not working, added zeros to average.")
            avg_beat = np.zeros(10)
            avg_rel = np.zeros(10)
            all_beat = np.zeros(10)
            all_rel = np.zeros(10)
        steps_beats2.append(avg_beat)
        steps_rels2.append(avg_rel)
    data_CPSC = scipy.io.loadmat("./tmp/CPSC_avgbeats_AF.mat")
    steps_beats1_CPSC = data_CPSC["steps_beats1"]
    steps_beats2_CPSC = data_CPSC["steps_beats2"]
    steps_rels1_CPSC = data_CPSC["steps_rels1"]
    steps_rels2_CPSC = data_CPSC["steps_rels2"]
    # zeropadding for same sample size
    max_len_CPSC = np.max([max(len(x) for x in steps_beats1_CPSC),max(len(x) for x in steps_beats2_CPSC)])
    max_len = np.max([max(len(x) for x in steps_beats1),max(len(x) for x in steps_beats2)])
    for i,step in enumerate(steps_beats1):
        pad1 = np.zeros(math.floor((max_len-len(step))/2))
        pad2 = np.zeros(math.ceil((max_len-len(step))/2))
        steps_beats1[i] = np.concatenate([pad1, step, pad2])
    for i,step in enumerate(steps_beats2):
        pad1 = np.zeros(math.floor((max_len-len(step))/2))
        pad2 = np.zeros(math.ceil((max_len-len(step))/2))
        steps_beats2[i] = np.concatenate([pad1, step, pad2])
    for i,step in enumerate(steps_rels1):
        pad1 = np.zeros(math.floor((max_len-len(step))/2))
        pad2 = np.zeros(math.ceil((max_len-len(step))/2))
        steps_rels1[i] = np.concatenate([pad1, step, pad2])
    for i,step in enumerate(steps_rels2):
        pad1 = np.zeros(math.floor((max_len-len(step))/2))
        pad2 = np.zeros(math.ceil((max_len-len(step))/2))
        steps_rels2[i] = np.concatenate([pad1, step, pad2])
    for i,step in enumerate(steps_beats1_CPSC):
        pad1 = np.zeros(math.floor((max_len_CPSC-len(step))/2))
        pad2 = np.zeros(math.ceil((max_len_CPSC-len(step))/2))
        steps_beats1_CPSC[i] = np.concatenate([pad1, step, pad2])
    for i,step in enumerate(steps_beats2_CPSC):
        pad1 = np.zeros(math.floor((max_len_CPSC-len(step))/2))
        pad2 = np.zeros(math.ceil((max_len_CPSC-len(step))/2))
        steps_beats2_CPSC[i] = np.concatenate([pad1, step, pad2])
    for i,step in enumerate(steps_rels1_CPSC):
        pad1 = np.zeros(math.floor((max_len_CPSC-len(step))/2))
        pad2 = np.zeros(math.ceil((max_len_CPSC-len(step))/2))
        steps_rels1_CPSC[i] = np.concatenate([pad1, step, pad2])
    for i,step in enumerate(steps_rels2_CPSC):
        pad1 = np.zeros(math.floor((max_len_CPSC-len(step))/2))
        pad2 = np.zeros(math.ceil((max_len_CPSC-len(step))/2))
        steps_rels2_CPSC[i] = np.concatenate([pad1, step, pad2])
    # calculate mean for every sample
    all_beats1_CPSC = np.mean(data_CPSC["steps_beats1"], axis=0)
    all_beats2_CPSC = np.mean(data_CPSC["steps_beats2"], axis=0)
    all_rels1_CPSC = np.mean(data_CPSC["steps_rels1"], axis=0)
    all_rels2_CPSC = np.mean(data_CPSC["steps_rels2"], axis=0)
    all_beats1 = np.mean(steps_beats1, axis=0)
    all_beats2 = np.mean(steps_beats2, axis=0)
    all_rels1 = np.mean(steps_rels1, axis=0)
    all_rels2= np.mean(steps_rels2, axis=0)
    # normalize to -1,1
    max_beats_CPSC = np.max([np.max(np.abs(all_beats1_CPSC)),np.max(np.abs(all_beats2_CPSC))])
    max_rels_CPSC = np.max([np.max(np.abs(all_rels1_CPSC)),np.max(np.abs(all_rels2_CPSC))])
    all_beats1_CPSC /= max_beats_CPSC
    all_rels1_CPSC /= max_rels_CPSC
    all_beats2_CPSC /= max_beats_CPSC
    all_rels2_CPSC /= max_rels_CPSC
    max_beats = np.max([np.max(np.abs(all_beats1)),np.max(np.abs(all_beats2))])
    max_rels = np.max([np.max(np.abs(all_rels1)),np.max(np.abs(all_rels2))])
    all_beats1 /= max_beats
    all_rels1 /= max_rels
    all_beats2 /= max_beats
    all_rels2 /= max_rels
    ##### plot raw data and relevances #####
    plt.rcParams.update({'font.size': 18})
    fig, axs = plt.subplots(1,2,figsize = (24, 8))
    factor = 5 # upsampling factor for relevances in scatter plot
    # first patientRange
    plt.subplot(1,2,1)
    labels = np.arange(0, max_len_CPSC/400, 0.1)
    resampled = signal.resample_poly(all_rels1, factor, 1)
    resampled2 = signal.resample_poly(all_rels1_CPSC, factor, 1)
    padding = (max_len_CPSC-len(all_beats1))/2
    padding2 = (max_len_CPSC-len(all_beats1_CPSC))/2
    x_range = np.arange(padding, len(all_beats1)+padding, 1)
    x_range2 = np.arange(padding2, len(all_beats1_CPSC)+padding2, 1)
    sigma1 = np.var(steps_rels1, axis=0)/max_rels
    plt.plot(np.arange(padding2, len(resampled2)/factor+padding2, 1/factor), resampled2, c="#998ec3", linewidth=6)
    plt.plot(np.arange(padding, len(resampled)/factor+padding, 1/factor), resampled, c="#f1a340", linewidth=6)
    plt.xticks(np.arange(0, max_len_CPSC, 400*0.1), [str(round(float(label), 2)) for label in labels])
    #plt.yticks(fontsize=12)
    plt.ylim(-1.05,1.05)
    plt.xlim(0,max_len_CPSC)
    plt.xlabel("seconds")
    plt.legend(["CPSC AF","PTB-XL AF"])
    plt.grid()
    # second patientRange
    plt.subplot(1,2,2)
    labels = np.arange(0, max_len_CPSC/400, 0.1)
    resampled = signal.resample_poly(all_rels2, factor, 1)
    resampled2 = signal.resample_poly(all_rels2_CPSC, factor, 1)
    padding = (max_len_CPSC-len(all_beats2))/2
    padding2 = (max_len_CPSC-len(all_beats2_CPSC))/2
    x_range = np.arange(padding, len(all_beats2)+padding, 1)
    x_range2 = np.arange(padding2, len(all_beats2_CPSC)+padding2, 1)
    sigma2 = np.var(steps_rels2, axis=0)/max_rels
    plt.plot(np.arange(padding2, len(resampled2)/factor+padding2, 1/factor), resampled2, c="#998ec3", linewidth=6)
    plt.plot(np.arange(padding, len(resampled)/factor+padding, 1/factor), resampled, c="#f1a340", linewidth=6)
    plt.xticks(np.arange(0, max_len_CPSC, 400*0.1), [str(round(float(label), 2)) for label in labels])
    plt.ylim(-1.05,1.05)
    plt.xlim(0,max_len_CPSC)
    plt.xlabel("seconds")
    plt.legend(["CPSC Normal","PTB-XL Normal"])
    plt.grid()
    # save figure
    plt.tight_layout()
    plt.savefig('./img/beats_comp_' + name + '.pdf')
    #scipy.io.savemat("./tmp/CPSC_avgbeats.mat", mdict={'steps_beats1': steps_beats1, 'steps_beats2': steps_beats2, 'steps_rels1': steps_rels1, 'steps_rels2': steps_rels2})
    plt.close()
    


In [ ]:
plotAvgsLeadImage(range(200), range(len(i_abnorm),len(i_abnorm)+200), chan_list[6] + abnorm_label, 6) # full size
#plotAvgsLeadImage(range(0,20), range(200,220), chan_list[4] + abnorm_label, 4) # small size for testing

In [ ]:
plotAvgsLeadImageComparison(range(200), range(len(i_abnorm),len(i_abnorm)+200), chan_list[6] + abnorm_label, 6) 

### Video: Average Heartbeat for each ECG as PNG
Code to convert PNGs to video with ffmpeg (https://ffmpeg.org/):

`ffmpeg -r 5 -f image2 -s 1920x1080 -i beats_V1AF_%03d.png -vcodec libx264 -crf 15 -filter_complex "color=white,format=rgb24[c];[c][0]scale2ref[c][i];[c][i]overlay=format=auto:shortest=1,setsar=1" -pix_fmt yuv420p beats_V1AF_single.mp4`

`ffmpeg -r 5 -f image2 -s 1920x1080 -i beats_AVLLBBB_%03d.png -vcodec libx264 -crf 15 -filter_complex "color=white,format=rgb24[c];[c][0]scale2ref[c][i];[c][i]overlay=format=auto:shortest=1,setsar=1" -pix_fmt yuv420p beats_AVLLBBB_single.mp4`

In [ ]:
def plotAvgsLeadVideo(patientRange, name, lead, plotRange):
    all_beats1 = []
    all_rels1 = []
    steps_beats1 = []
    steps_rels1 = []
    # first patientRange
    all_avg_beats = np.zeros(200)
    all_avg_rels = np.zeros(200)
    for patient in tqdm(patientRange):
        try:
            max_avg_beat = 0
            max_avg_rel = 0
            max_all_beat = 0
            max_all_rel = 0
            for lead2 in range(12):
                avg_beat, avg_rel, all_beat, all_rel, duration = getPatientAvgs(patient, lead2)
                max_avg_beat = np.max([np.max(np.abs(avg_beat)), max_avg_beat])
                max_avg_rel = np.max([np.max(np.abs(avg_rel)), max_avg_rel])
                max_all_beat = np.max([np.max(np.abs(all_beat)), max_all_beat])
                max_all_rel = np.max([np.max(np.abs(all_rel)), max_all_rel])
            avg_beat, avg_rel, all_beat, all_rel, duration = getPatientAvgs(patient, lead)
            # normalize to -1,1
            avg_beat /= max_avg_beat
            avg_rel /= max_avg_rel
            all_beat /= max_all_beat
            all_rel /= max_all_rel
        except:
            print(pid[patient] + " and lead " + str(lead) + " not working, added zeros to average.")
            avg_beat = np.zeros(10)
            avg_rel = np.zeros(10)
            all_beat = np.zeros(10)
            all_rel = np.zeros(10)
        # copy shorter beat on bigger one
        all_avg_beats, all_avg_rels = addAvgs(all_avg_beats, all_avg_rels, avg_beat, avg_rel)
        if patient in plotRange:
            steps_beats1.append(avg_beat)
            steps_rels1.append(avg_rel)
    all_beats1 = all_avg_beats
    all_rels1 = all_avg_rels
    # normalize to -1,1
    max_beats = np.max(np.abs(all_beats1))
    max_rels = np.max(np.abs(all_rels1))
    all_beats1 /= max_beats
    all_rels1 /= max_rels
    max_len = len(all_beats1)
    factor = 5 # upsampling factor for relevances in scatter plot
    # sort recordings for confidence
    sortedConf = np.argsort([item[abnorm] for item in conf])
    ##### plot raw data and relevances #####
    for step, i in enumerate(sortedConf):
        fig = plt.figure(figsize = (12, 6))
        max_beats = np.max(np.abs(steps_beats1[i]))
        max_rels = np.max(np.abs(steps_rels1[i]))
        #padding = (max_len-len(steps_beats1[i]))/2
        padding = 0
        resampled = signal.resample_poly(steps_rels1[i]/max_rels, factor, 1)
        x_range = np.arange(padding, len(steps_beats1[i])+padding, 1)
        plt.plot(x_range, steps_beats1[i]/max_beats, color="black", label="ECG")
        plt.scatter(np.arange(padding, len(resampled)/factor+padding, 1/factor), resampled, c=resampled, cmap=cm.coolwarm, s=20, vmin=-1, vmax=1)
        plt.title("Lead " + chan_list[lead] + "\n Recording: " + str(step+1).zfill(3), loc='right', fontsize=15)
        plt.text(0, 1, 'DNN: ' + (abnorm_label if conf[i][abnorm] >= threshold[abnorm] else "no " + abnorm_label) + " (" + str(conf[i][abnorm]) + ")", horizontalalignment='left', verticalalignment='bottom', fontsize=15, color=("red" if (conf[i][abnorm] < threshold[abnorm] and i < 200) else 'black'))
        plt.text(max_len/2, 1.25, "Ground Truth: " + (abnorm_label if i < 200 else "Normal ECG"), horizontalalignment='center', verticalalignment='bottom', fontsize=18, color=("red" if (conf[i][abnorm] < threshold[abnorm] and i < 200) else 'black'))
        labels = np.arange(0, len(steps_beats1[i])/400, 0.1)
        plt.xticks(np.arange(0, len(steps_beats1[i]), 400*0.1), [str(round(float(label), 2)) for label in labels], fontsize=12)
        plt.yticks(fontsize=12)
        plt.ylim(-1,1)
        plt.xlim(0,len(steps_beats1[i]))
        plt.xlabel("seconds")
        plt.legend()
        plt.tight_layout()
        plt.savefig('./img/beats_' + name + "_" + str(step).zfill(3) + '.pdf')
        plt.close()

In [ ]:
plotAvgsLeadVideo(range(400), chan_list[6] + abnorm_label, 6, range(400))

## XAI Method Comparison

In [ ]:
patient = 23
lead = 11
methods = ["eps", "ab0", "wsq", "psa", "igr"]
batchsize = 1
relevances = []
file = "./tmp/200each_AF_"

plt.rcParams.update({'font.size': 18})

for method in methods:
    if method == "igr":
        max_rel = np.max(np.abs(data['rel_linear'][patient][:,lead]))
        relevances.append(data['rel_linear'][patient]/max_rel)
        continue
    data2 = scipy.io.loadmat(file + method + ".mat")
    max_rel = np.max(np.abs(data2['rel_linear'][patient][:,lead]))
    relevances.append(data2['rel_linear'][patient]/max_rel)
    
viz.plotXAIMethodComparison(data['raw'][patient], "linear", relevances, lead, methods, data['pid'][patient])

# MISC

## Zip images for easier download

In [ ]:
import zipfile
import pathlib
directory = pathlib.Path("./img")

with zipfile.ZipFile("images.zip", mode="w") as archive:
    for file_path in directory.iterdir():
        archive.write(file_path, arcname=file_path.name)

## DNN Results

In [ ]:
# plot model output
viz.plotClassBoxplot(conf[:int(numberSubjects/2)], conf[int(numberSubjects/2):], abnorm_label, 'Normal')

## Observer confidence vs DNN confidence

In [ ]:
cardio1 = np.genfromtxt('./data/cardiologist1.csv',delimiter=',')
cardio2 = np.genfromtxt('./data/cardiologist2.csv',delimiter=',')
cardiores = np.genfromtxt('./data/cardiology_residents.csv',delimiter=',')
erres = np.genfromtxt('./data/emergency_residents.csv',delimiter=',')
medstud = np.genfromtxt('./data/medical_students.csv',delimiter=',')

conf_cardio1 = [cardio1[val+1][abnorm] for i,val in enumerate(pid)]
conf_cardio2 = [cardio2[val+1][abnorm] for i,val in enumerate(pid)]
conf_cardiores = [cardiores[val+1][abnorm] for i,val in enumerate(pid)]
conf_erres = [erres[val+1][abnorm] for i,val in enumerate(pid)]
conf_medstud = [medstud[val+1][abnorm] for i,val in enumerate(pid)]
conf_observer = [(conf_cardio1[i]+conf_cardio2[i]+conf_cardiores[i]+conf_erres[i]+conf_medstud[i])/5 for i in range(numberSubjects)]

np.savetxt('./tmp/observer_conf_' + abnorm_label + '.txt', conf_observer, delimiter=',')
np.savetxt('./tmp/dnn_conf_' + abnorm_label + '.txt', conf[:,abnorm], delimiter=',')

print(np.corrcoef(conf_observer, conf[:,abnorm]))

In [ ]:
# only working with AF data loaded

dnn_af = np.loadtxt("./tmp/dnn_conf_AF.txt")
dnn_lbbb = np.loadtxt("./tmp/dnn_conf_LBBB.txt")
observer_af = np.loadtxt("./tmp/observer_conf_AF.txt")
observer_lbbb = np.loadtxt("./tmp/observer_conf_LBBB.txt")

plt.figure(figsize = (6, 6))
plt.title("Confidence for Ribeiro records\nPearson Correlation: " + str(np.corrcoef(np.concatenate([dnn_af, dnn_lbbb]), np.concatenate([observer_af, observer_lbbb]))[0][1]))
plt.scatter(dnn_af, observer_af, color="#386cb0", label="AF")
plt.scatter(dnn_lbbb, observer_lbbb, color="#fdc086", label="LBBB")
plt.scatter(dnn_af[[(i) for i in range(len(dnn_af)) if i not in abnorm_pos]], observer_af[[(i) for i in range(len(dnn_af)) if i not in abnorm_pos]], label="False Negative ", s=20, color="red", marker="x")
plt.ylim(-0.1,1.1)
plt.xlim(-0.1,1.1)
plt.xlabel("DNN Confidence")
plt.ylabel("Average Observer Confidence")
plt.legend()
plt.axline((0, 0), slope=1, color="black", linestyle=(0, (5, 5)))
plt.tight_layout()
plt.grid()
plt.savefig('./img/observer_conf.pdf')
plt.show()